# Retrieval-Augmented Generation (RAG)

In [1]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [2]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [3]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 30.4MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [10]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [5]:
# calorie_lookup_tool('bananas')

In [25]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool]
)

In [12]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

Total ~200 kcal (based on typical sizes: banana ~118 g and apple ~182 g).

- Banana: 89 kcal per 100 g
  - ~105 kcal for ~118 g
- Apple: 52 kcal per 100 g
  - ~95 kcal for ~182 g

Notes: results are approximate depending on actual fruit sizes.


In [ ]:
nutrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [29]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function to look up nutrition information.

    Args:
        query: The question to look up.
        max_results: The maximum number of results to return.
    Returns:
        A string containing and answer to the nutrition question.
    """
    
    results = nutrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)
    
    return "Here are questions and answers related to your question:\n" + "\n".join(formatted_results)

In [35]:
new_calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If you are asked about nutrition in general, use the nutrition_qna_tool first
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool]
)

In [36]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        new_calorie_agent,
        "Are bananas healthy? Provide some calorie information as well.",
    )
    print(result.final_output)

Bananas are generally healthy and provide nutrients like potassium, vitamin B6, and fiber. They fit well in a balanced diet.

Calorie info (per 100 g):
- Banana: 89 kcal
- Banana juice: 50 kcal
- Banana (tropical/exotic variety): 89 kcal

Note: Serving sizes vary; a medium banana (~118 g) is about 105 kcal.
